In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time 
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import os
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import pandas as pd
import numpy as np
import shutil

import import_ipynb

# CLICK RUN ALL
### Press Control + Enter ot run ALL cells

## THE REAL DEAL!!

In [7]:
def create_and_update_individual(menu):
    start = time.time()
    dirName = "K:\\BoT\\"
    dirName2 = "K:\\BoT\\Reports ID reference\\"
    if not os.path.exists(dirName2):
        os.makedirs(dirName2)
    chromeOptions = webdriver.ChromeOptions()
    prefs = {"download.default_directory" : dirName}
    chromeOptions.add_experimental_option("prefs",prefs)
    chromedriver = "./chromedriver"
    driver = webdriver.Chrome(executable_path=chromedriver, chrome_options=chromeOptions)
    driver.get("https://www.bot.or.th/English/Statistics/Pages/default.aspx")    
    # Navigate through the page
    section = []
    id_list = []
    name_list = []
    df = pd.DataFrame()
    
    link = driver.find_element_by_link_text(menu)
    link.click()      

    # lists for id charts


    hRefs = []

    # Find the links and get all the names
    parent = driver.find_elements_by_class_name("bullet_container")
    for link in parent:
        links = link.find_elements_by_tag_name('a')
        for l in links:
            hRefs.append(str(l.text))

    # Click all the links by looping over the names that were collected
    for text in hRefs:
        link = driver.find_element_by_link_text(text)
        link.click()
        href = []
        get_id = False
        elems = driver.find_elements_by_xpath("//a[@href]")
        size = len(elems)

#         print('Folder: ' + text)
        # Traverse and download all files that is possible
        while size > 0:
            elems = driver.find_elements_by_xpath("//a[@href]")
            try:
                link = elems[size - 1].get_attribute('href')

                # Check for an excel file
                if str(link).endswith('XLSX'):
                    get_id = True
                    name = elems[size - 1].text
                    href.append(link)            
                    elems[size - 1].click()
                size = size - 1
            except Exception as ex:
                template = "An exception of type {0} occurred. Arguments:\n{1!r}"
                message = template.format(type(ex).__name__, ex.args)
#                 print(message)
                driver.back()
                size = size - 1
        if get_id:
            Id = driver.find_elements_by_class_name("tx-news")
#             print('id size:', len(Id))
            table = driver.find_element_by_class_name("table")
            anchor = table.find_elements_by_tag_name("a")

            size = 0

            for i in anchor:
                cur = i.text
                if cur != '':
                    name_list.append(cur)
                    size = size + 1
#             print('name size:' , size)
            for i in range(size):
                cur = Id[i].text
                id_list.append(cur)
                section.append(menu)
        driver.back()


    df['section'] = section
    df['report_id'] = id_list
    df['report_name'] = name_list
    #     df.to_csv('Reports ID Chart Individual.csv')
    df2 = pd.read_csv(dirName2 + 'ID Reference Chart.csv', index_col = 0)
    df2 = df2[df2.section != menu]
    df2 = df2.append(df)
    df2 = df2.reset_index(drop=True)
    df2.to_csv(dirName2 + 'ID Reference Chart.csv')
    end = time.time()
    driver.close()
    print('time execute: ', end - start)

def create_and_update_bot_all():
    start = time.time()
    dirName = "K:\\BoT\\"
    dirName2 = "K:\\BoT\\Reports ID reference\\"
    if os.path.exists(dirName):
        shutil.rmtree(dirName)
    if not os.path.exists(dirName2):
        os.makedirs(dirName2)
    chromeOptions = webdriver.ChromeOptions()
    menu_list = ['Financial Markets', 'Economic and Financial Statistics', 'Financial Institutions Statistics', 'Payment Systems', 'Regional Economic and Financial Statistics']
    prefs = {"download.default_directory" : dirName}
    chromeOptions.add_experimental_option("prefs",prefs)
    chromedriver = "./chromedriver"
    driver = webdriver.Chrome(executable_path=chromedriver, chrome_options=chromeOptions)
    driver.get("https://www.bot.or.th/English/Statistics/Pages/default.aspx")    
    # Navigate through the page
    
    id_list = []
    name_list = []
    section = []
    df = pd.DataFrame()
    
    for menu in menu_list:
        link = driver.find_element_by_link_text(menu)
        link.click()      

        # lists for id charts
        


        hRefs = []

        # Find the links and get all the names
        parent = driver.find_elements_by_class_name("bullet_container")
        for link in parent:
            links = link.find_elements_by_tag_name('a')
            for l in links:
                hRefs.append(str(l.text))

        # Click all the links by looping over the names that were collected
        for text in hRefs:
            link = driver.find_element_by_link_text(text)
            link.click()
            href = []
            get_id = False
            elems = driver.find_elements_by_xpath("//a[@href]")
            size = len(elems)

    #         print('Folder: ' + text)
            # Traverse and download all files that is possible
            while size > 0:
                elems = driver.find_elements_by_xpath("//a[@href]")
                try:
                    link = elems[size - 1].get_attribute('href')

                    # Check for an excel file
                    if str(link).endswith('XLSX'):
                        get_id = True
                        name = elems[size - 1].text
                        href.append(link)            
                        elems[size - 1].click()
                    size = size - 1
                except Exception as ex:
                    template = "An exception of type {0} occurred. Arguments:\n{1!r}"
                    message = template.format(type(ex).__name__, ex.args)
    #                 print(message)
                    driver.back()
                    size = size - 1
            if get_id:
                Id = driver.find_elements_by_class_name("tx-news")
    #             print('id size:', len(Id))
                table = driver.find_element_by_class_name("table")
                anchor = table.find_elements_by_tag_name("a")

                size = 0

                for i in anchor:
                    cur = i.text
                    if cur != '':
                        name_list.append(cur)
                        size = size + 1
    #             print('name size:' , size)
                for i in range(size):
                    cur = Id[i].text
                    id_list.append(cur)
                    section.append(menu)
            driver.back()
        driver.back()
        time.sleep(10)

    

    df['section'] = section
    df['report_id'] = id_list
    df['report_name'] = name_list
    driver.close()
    df.to_csv(dirName2 + 'ID Reference Chart.csv')
    
    end = time.time()
    print('time execute: ', end - start)

# def remove_dups:
    
        

In [8]:
dirName = "K:\\BoT\\"
def remove_rename_dup():
    list_of_update = []
    for filename in os.listdir("K:\\BoT\\"):
        if 'ENG_ALL' in filename:
            name = filename[:-13] + '.XLSX'
            list_of_update.append(name)

# Find if there is a file that contains those name -- then delete the file
    i = 0
    for filename in os.listdir("K:\\BoT\\"):
        for file in list_of_update:
            if filename == file:  
                os.remove("K:\\BoT\\" + filename)

    for filename in os.listdir("K:\\BoT\\"):
        if 'ENG_ALL' in filename:
            name = filename[:-13] + '.XLSX'
            src = "K:\\BoT\\" + filename
            name = "K:\\BoT\\" + name
            os.rename(src, name)

def change_eng_all():
    for filename in os.listdir("K:\\BoT\\"):
        if '(' in filename:
            os.remove("K:\\BoT\\" + filename)


    for filename in os.listdir("K:\\BoT\\"):
        if 'ENG_ALL' in filename:
            name = filename[:-13] + '.XLSX'
            src = "K:\\BoT\\" + filename
            name = "K:\\BoT\\" + name
            os.rename(src, name)

In [11]:
prompt = input('''Select numbers from the following: 
[1] Financial Markets 
[2] Economic and Financial Statistics 
[3] Financial Institutions Statistics
[4] Payment Systems
[5] Regional Economic and Financial Statistics
[6] ALL WARNING: This could take up to 30 minutes
--> ''')

start = time.time()

menu_list = ['Financial Markets', 'Economic and Financial Statistics', 'Financial Institutions Statistics', 'Payment Systems', 'Regional Economic and Financial Statistics']
if prompt == '6':
    create_and_update_bot_all()
    change_eng_all()
elif int(prompt) < 6:
    create_and_update_individual(menu_list[int(prompt) - 1])
    remove_rename_dup()
else:
    print('Invalid number: try again')
    pass


Select numbers from the following: 
[1] Financial Markets 
[2] Economic and Financial Statistics 
[3] Financial Institutions Statistics
[4] Payment Systems
[5] Regional Economic and Financial Statistics
[6] ALL WARNING: This could take up to 30 minutes
--> 8
Invalid number: try again


In [ ]:
# Change file name


In [ ]:
# list_of_removed = []
# for filename in os.listdir("K:\\BoT\\"):
#     if '(' in filename:
#         os.remove("K:\\BoT\\" + filename)
        

# for filename in os.listdir("K:\\BoT\\"):
#     if 'ENG_ALL' in filename:
#         name = filename[:-13] + '.XLSX'
#         src = "K:\\BoT\\" + filename
#         name = "K:\\BoT\\" + name
#         os.rename(src, name)
#         os.close(name)

In [ ]:
# menu = 'Financial Markets'
# dirName = "K:\\Bank of Thailand Stats\\" + menu
# if os.path.exists(dirName):
#     shutil.rmtree(dirName)